In [11]:
import sys, os
sys.path.append("../../")
from carlasim.carla_client import CarlaClient
from carlasim.carla_ego_car import CarlaEgoCar
from carlasim.sensors.data_sensors import *
from scenario_builder import ScenarioBuilder, ScenarioActor
from data_logger import DataLogger
from vision.occupancy_grid_cuda import OccupancyGrid
from model.waypoint import Waypoint
import cv2

client = CarlaClient(town='Town07')
ego = CarlaEgoCar(client)
ego.init_fake_bev_seg_camera()
ego.set_pose(0, 0, 0, 0)
ego.set_brake(1.0)

sb = ScenarioBuilder(client)

counter = 0

def save_planning_data(ego: CarlaEgoCar, counter: int) -> int:
    logger = DataLogger(0, ego, f"imgs/log_{counter + 1}.log")
    bev = ego.get_bev_camera().read()
    logger.sample(counter + 1)
    bev = OccupancyGrid(bev, 0, 0, Waypoint(0,0), Waypoint(0,0)).get_color_frame()
    cv2.imwrite(f"imgs/bev_{counter + 1}.png", bev)
    logger.destroy()
    return counter + 1

#### Path planning dataset build:
- <b>carla client</b>: client
- <b>ego car</b>: ego

In [12]:
location = [-70, 0, 0, 0]

In [13]:
ego.set_pose(location[0], location[1], location[2], location[3])


In [16]:
sb.load_scenario("scenarios/cars_zigzag.sce")


TypeError: MapPose.__init__() got an unexpected keyword argument 'heading_angle'

#### Save current state

In [15]:
counter = save_planning_data(ego, counter)